# OpenAI Agents SDK - Brief Demo

Quick demonstration of four key concepts in the OpenAI Agents SDK:
1. **Agents** - Defining and running specialized AI agents
2. **Handoffs** - Agents transferring control to others
3. **Guardrails** - Input/output validation
4. **Sessions** - Stateful conversation history

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, SQLiteSession
import asyncio

# Load environment variables
load_dotenv(override=True)

print("✅ OpenAI Agents SDK loaded")

✅ OpenAI Agents SDK loaded


## 1. Agents - Define and Run Specialized AI Agents

In [2]:
# Create specialized agents with specific instructions
math_agent = Agent(
    name="Math Agent",
    instructions="You solve math problems step by step. Be concise."
)

history_agent = Agent(
    name="History Agent",
    instructions="You answer history questions with dates and context. Be concise."
)

# Actually run the agents
async def demo_agents():
    # Test math agent
    math_result = await Runner.run(math_agent, "What is 5 + 3?")
    print(f"Math Agent: {math_result.final_output}\n")

    # Test history agent
    history_result = await Runner.run(history_agent, "When was World War II?")
    print(f"History Agent: {history_result.final_output}")

await demo_agents()

Math Agent: 5 + 3 = 8

History Agent: World War II took place from September 1, 1939, when Germany invaded Poland, to September 2, 1945, when Japan formally surrendered. It was the deadliest conflict in human history, involving most of the world's nations.


## 2. Handoffs - Agents Transfer Control

In [3]:
# Triage agent can hand off to specialized agents
triage_agent = Agent(
    name="Triage Agent",
    instructions="Route math questions to Math Agent, history to History Agent. Briefly say who you're routing to.",
    handoffs=[math_agent, history_agent]
)

# Demonstrate actual handoff
async def demo_handoff():
    result = await Runner.run(triage_agent, "What is 10 divided by 2?")
    print(result.last_agent)
    print(f"Triage: {result.final_output}")
await demo_handoff()

# Demonstrate actual handoff
async def demo_handoff2():
    result = await Runner.run(triage_agent, "When did Singapore gain independence?")
    print(result.last_agent)
    print(f"Triage: {result.final_output}")
await demo_handoff2()

Agent(name='Math Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You solve math problems step by step. Be concise.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)
Triage: 10 divided by 2 is:

10 ÷ 2 = 5

Final answer: 5
Agent(name='History Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You answer history questions with dates and context. Be concise.', prompt=None, handoffs=[], model=None, model_settings=ModelSet

## 3. Guardrails - Input Validation (Working Demo)

In [4]:
# Create an agent with validation built into instructions
# This demonstrates guardrail concept through agent behavior
restricted_agent = Agent(
    name="Restricted Agent",
    instructions="""You ONLY answer math or history questions.
    If asked about anything else (weather, sports, food, etc.), respond:
    'I can only help with math or history questions.'"""
)

async def demo_guardrails():
    print("Testing input validation through agent behavior:\n")

    # Valid: Math question
    result = await Runner.run(restricted_agent, "What is 5 + 3?")
    print(f"✅ Math question: {result.final_output}\n")

    # Valid: History question
    result = await Runner.run(restricted_agent, "When was the American Revolution?")
    print(f"✅ History question: {result.final_output}\n")

    # Invalid: Weather question (agent should refuse)
    result = await Runner.run(restricted_agent, "What's the weather today?")
    print(f"❌ Weather question: {result.final_output}")

await demo_guardrails()

Testing input validation through agent behavior:

✅ Math question: 5 + 3 = 8

✅ History question: The American Revolution took place from 1775 to 1783. It was a conflict between the thirteen American colonies and Great Britain, leading to the independence of the United States.

❌ Weather question: I can only help with math or history questions.


## 4. Sessions - Stateful Conversation History

In [5]:
# Sessions maintain conversation history across interactions
async def demo_sessions():
    # Create a session for maintaining state
    session = SQLiteSession("demo_conversation")

    print("With Sessions - Stateful conversation:\n")

    # First interaction
    result1 = await Runner.run(
        math_agent,
        "Calculate 4 * 4",
        session=session
    )
    print(f"Q1: Calculate 4 * 4")
    print(f"A1: {result1.final_output}\n")

    # Second interaction - agent remembers previous context
    result2 = await Runner.run(
        math_agent,
        "Now multiply that result by 3",
        session=session
    )
    print(f"Q2: Now multiply that result by 3")
    print(f"A2: {result2.final_output}")
    print("\n✅ Session maintains conversation history!")

await demo_sessions()

With Sessions - Stateful conversation:

Q1: Calculate 4 * 4
A1: 4 * 4 = 16

Q2: Now multiply that result by 3
A2: 16 × 3 = 48

✅ Session maintains conversation history!


## Summary

**OpenAI Agents SDK** demonstrated:
- **Agents**: Specialized AI agents processing real requests
- **Handoffs**: Multi-agent workflows with control transfer
- **Guardrails**: Input validation through agent behavior
- **Sessions**: Stateful conversations with automatic history management using SQLiteSession

Each concept was shown with actual running code demonstrating real agent behavior.